In [65]:
import os

In [66]:
%pwd

'c:\\Users\\azfar\\Downloads\\wine-ml-proj\\wine-ml'

In [67]:
os.chdir("../")

In [68]:
os.chdir("wine-ml-proj")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'wine-ml-proj'

In [69]:
%pwd

'c:\\Users\\azfar\\Downloads\\wine-ml-proj'

In [70]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [71]:
from wineml.constants import *
from wineml.utils.common import read_yaml, create_directories

In [72]:
from pathlib import Path
from src.wineml.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.wineml.utils.common import read_yaml, create_directories
from src.wineml.entity.config_entity import DataIngestionConfig
from box import ConfigBox


class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH
    ):
        self.config: ConfigBox = read_yaml(config_filepath)
        self.params: ConfigBox = read_yaml(params_filepath)
        self.schema: ConfigBox = read_yaml(schema_filepath)

        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config


In [73]:
import os
import urllib.request as request
import zipfile
from wineml.logging.logger import logger
from wineml.utils.common import get_size

In [74]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists — size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the downloaded zip file into the unzip directory.
        Returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        logger.info(f"Data extracted to: {unzip_path}")


In [75]:
import os
os.chdir("wine-ml")

In [77]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-06 22:50:11,088]: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-06 22:50:11,090]: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-06 22:50:11,094]: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-06 22:50:11,095]: INFO: common: created directory at: artifacts]
[2025-05-06 22:50:11,097]: INFO: common: created directory at: artifacts\data_ingestion]
[2025-05-06 22:50:12,054]: INFO: 1092207118: artifacts\data_ingestion\data.zip downloaded! with following info: 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A76E:9D8CE:660B1C:78B3F5:681ACA62
Accept-Ranges: bytes
Date: 